
# ECE GPT — Banka Yaşayan Sistem Simülasyonu (Q–W–K–V–S + Process/System)

Bu not defteri, **Ece GPT (q,w,k,v,s) attention** kullanan küçük bir dil modeli ile,
**banka sistemini yaşayan bir varlık** olarak simüle eder:

- **EPS veri seti** (Event + Process + System) üretimi ve satıra açma (linearization)
- **EceGPT** (decoder-only, q,w,k,v,s attention) eğitimi (hızlı demo)
- **Model tabanlı üretim**: `<EVQ> Q </EVQ> <EVW> W </EVW> <EVK>` prompt'u ile **K, V, S**'yi model üretir
- **Banka state makinesi**: S (yeni durum) → metrikleri günceller, tetikleme ile bir sonraki **Q** seçilir
- **Grafik Rapor**: sermaye, likidite, NIM, NPL, güven zaman serileri

> Not: Demo amaçlıdır; eğitim kısa tutulmuştur.


In [ ]:

import math, random, os, json, time
from dataclasses import dataclass, asdict
from typing import Dict, List, Tuple

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import pandas as pd
import matplotlib.pyplot as plt

random.seed(42); torch.manual_seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device


## 1) EPS (Event+Process+System) veri seti üretimi

In [ ]:

Q_pool = [
    "Sermaye yeterliliği", "Likidite yönetimi", "Kredi büyümesi", "Faiz riski", "Piyasa riski",
    "Takipteki krediler", "Kredi notu", "Özkaynak karlılığı", "Mevduat tabanı", "Varlık kalitesi"
]
W_pool = [
    "kredi portföyü hızla genişlerse", "mevduat çıkışı hızlanırsa", "faizler düşerse", "faizler yükselirse",
    "ekonomik belirsizlik artarsa", "tahsilatlar yavaşlarsa", "döviz kuru sıçrarsa", "tahvil fiyatları düşerse"
]
K_pool = [
    "karşılık giderleri yükselirse", "kısa vadeli borçlanma maliyeti artarsa", "kredi talebi artarsa",
    "vade uyumsuzluğu büyürse", "türev pozisyonları zarara dönerse", "teminat değerleri gerilerse",
    "fonlama maliyeti artarsa", "sendikasyon yenilenmezse"
]
V_pool = [
    "sermaye oranı geriler", "likidite tamponu erir", "risk iştahı yükselir", "fonlama baskısı artar",
    "net faiz marjı daralır", "varlık kalitesi bozulur", "karlılık azalır", "kredi arzı kısılır"
]
S_pool = [
    "bankalar temkinli olur", "ödeme gücü riske girer", "kredi kalitesi bozulabilir", "faiz riski yönetimi sıkılaşır",
    "piyasa oynaklığına duyarlılık artar", "yeniden fiyatlama döngüsü hızlanır", "sermaye artırımı gündeme gelir",
    "bilançoda küçülmeye gidilir"
]

process_templates = [
    {"when":"Kredi talebi artar","how":"Banka düşük faizli krediler sunar","if":"Geri ödemeler aksarsa",
     "then":"Takipteki kredi oranı yükselir","so":"Karşılık giderleri artar","result":"Karlılık azalır","award":"Risk yönetimi puanı düşer"},
    {"when":"Mevduat çıkışı hızlanır","how":"Banka kısa vadeli borçlanmaya gider","if":"Borçlanma maliyeti artarsa",
     "then":"Likidite rezervi erir","so":"Ödeme güçlüğü riski doğar","result":"Güven kaybı yaşanır","award":"Kredi notu düşer"},
    {"when":"Faizler yükselir","how":"Banka sabit faizli varlıkları hedge eder","if":"Hedge etkinliği zayıflarsa",
     "then":"Net faiz marjı baskılanır","so":"Riskten korunma maliyeti artar","result":"Sermaye kârlılığı düşer","award":"Pozisyon limiti kısılır"},
    {"when":"Ekonomik belirsizlik artar","how":"Kredi standartları sıkılaştırılır","if":"Talep zayıflarsa",
     "then":"Kredi büyümesi yavaşlar","so":"Gelir artışı sınırlanır","result":"Maliyet disiplini zorunlu olur","award":"Verimlilik programı devreye alınır"}
]

system_templates = [
    {"body":"Ticari banka","life":"Yıllık faaliyet döngüsü","lifetime":"50 yıl",
     "input":"Mevduat, özkaynak, borçlanma","output":"Krediler, yatırım gelirleri, komisyon gelirleri",
     "components":"Şube ağı, dijital kanallar, insan kaynağı, IT altyapısı",
     "value":"Finansal istikrar, marka güveni, müşteri memnuniyeti"},
    {"body":"Yatırım bankası","life":"Çeyreklik faaliyet döngüsü","lifetime":"30 yıl",
     "input":"Kurumsal yatırımlar, tahvil ihraçları, yatırım fonları",
     "output":"Danışmanlık ücretleri, yatırım getirileri",
     "components":"Portföy yönetimi, risk analizi birimi, global ağ",
     "value":"Piyasa itibarı, yatırımcı getirisi, stratejik ortaklıklar"}
]

def make_event():
    import random
    return {"Q":random.choice(Q_pool),"W":random.choice(W_pool),"K":random.choice(K_pool),
            "V":random.choice(V_pool),"S":random.choice(S_pool)}

def make_sample():
    import random
    return {"event":make_event(),"process":random.choice(process_templates),"system":random.choice(system_templates)}

samples = [make_sample() for _ in range(60)]
len(samples)


## 2) Satıra açma (linearization), tokenizasyon ve Dataset

In [ ]:

TAGS = [
    "<EVQ>","</EVQ>","<EVW>","</EVW>","<EVK>","</EVK>","<EVV>","</EVV>","<EVS>","</EVS>",
    "<PRW>","</PRW>","<PRH>","</PRH>","<PRI>","</PRI>","<PRT>","</PRT>","<PRS>","</PRS>","<PRR>","</PRR>","<PRA>","</PRA>",
    "<SYB>","</SYB>","<SYL>","</SYL>","<SYT>","</SYT>","<SYI>","</SYI>","<SYO>","</SYO>","<SYC>","</SYC>","<SYV>","</SYV>",
    "<EOS>"
]
def linearize(s):
    e,p,y = s["event"], s["process"], s["system"]
    return (f"<EVQ> {e['Q']} </EVQ> <EVW> {e['W']} </EVW> <EVK> {e['K']} </EVK> "
            f"<EVV> {e['V']} </EVV> <EVS> {e['S']} </EVS> "
            f"<PRW> {p['when']} </PRW> <PRH> {p['how']} </PRH> <PRI> {p['if']} </PRI> "
            f"<PRT> {p['then']} </PRT> <PRS> {p['so']} </PRS> <PRR> {p['result']} </PRR> <PRA> {p['award']} </PRA> "
            f"<SYB> {y['body']} </SYB> <SYL> {y['life']} </SYL> <SYT> {y['lifetime']} </SYT> "
            f"<SYI> {y['input']} </SYI> <SYO> {y['output']} </SYO> <SYC> {y['components']} </SYC> <SYV> {y['value']} </SYV> <EOS>")

texts = [linearize(s) for s in samples]
train_texts, val_texts = texts[:50], texts[50:]

def tokenize(text:str):
    for t in TAGS: text = text.replace(t, f" {t} ")
    return [tok for tok in text.split() if tok.strip()]

vocab = sorted(set(tok for tx in (train_texts+val_texts) for tok in tokenize(tx)))
stoi = {s:i for i,s in enumerate(vocab)}; itos = {i:s for s,i in stoi.items()}
V = len(vocab)

def encode(text:str): return [stoi[s] for s in tokenize(text)]
def decode(ids): return " ".join(itos[i] for i in ids)

class LMDataset(Dataset):
    def __init__(self, texts):
        self.data = []
        for t in texts:
            ids = encode(t); self.data.append((ids[:-1], ids[1:]))
    def __len__(self): return len(self.data)
    def __getitem__(self, i):
        x,y = self.data[i]
        return torch.tensor(x, dtype=torch.long), torch.tensor(y, dtype=torch.long)

def collate(batch):
    xs,ys = zip(*batch)
    L = max(x.size(0) for x in xs)
    X = torch.zeros((len(xs), L), dtype=torch.long)
    Y = torch.zeros((len(ys), L), dtype=torch.long)
    for i,(x,y) in enumerate(zip(xs,ys)):
        X[i,:x.size(0)] = x; Y[i,:y.size(0)] = y
    return X.to(device), Y.to(device)

len(vocab), len(train_texts), len(val_texts)


## 3) EceGPT (q,w,k,v,s) — model

In [ ]:

class ECEAttention(nn.Module):
    def __init__(self, d_model=128, n_heads=4):
        super().__init__(); assert d_model % n_heads == 0
        self.h, self.dh = n_heads, d_model//n_heads
        self.Wq = nn.Linear(d_model, d_model, bias=False)
        self.Wk = nn.Linear(d_model, d_model, bias=False)
        self.Wv = nn.Linear(d_model, d_model, bias=False)
        self.Ww = nn.Linear(d_model, d_model, bias=False)
        self.Ws = nn.Linear(d_model, d_model, bias=False)
        self.lam = nn.Parameter(torch.zeros(n_heads))
        self.mu  = nn.Parameter(torch.zeros(n_heads))
        self.out = nn.Linear(d_model, d_model, bias=False)
    def split(self,x):
        B,T,C = x.size(); return x.view(B,T,self.h,self.dh).transpose(1,2)
    def join(self,x):
        B,H,T,Dh = x.size(); return x.transpose(1,2).contiguous().view(B,T,H*Dh)
    def forward(self,x):
        q=self.split(self.Wq(x)); k=self.split(self.Wk(x)); v=self.split(self.Wv(x))
        w=self.split(self.Ww(x)); s=self.split(self.Ws(x))
        lam=self.lam.view(1,self.h,1,1); mu=self.mu.view(1,self.h,1,1)
        k_aug = k + lam*w + mu*s
        scores = torch.matmul(q, k_aug.transpose(-2,-1)) / math.sqrt(self.dh)
        T = scores.size(-1); mask = torch.tril(torch.ones(T,T, device=scores.device)).view(1,1,T,T)
        scores = scores.masked_fill(mask==0, float("-inf"))
        attn = torch.softmax(scores, dim=-1)
        out = torch.matmul(attn, v)
        return self.out(self.join(out))

class Block(nn.Module):
    def __init__(self, d_model=128, n_heads=4):
        super().__init__()
        self.ln1 = nn.LayerNorm(d_model); self.attn = ECEAttention(d_model, n_heads)
        self.ln2 = nn.LayerNorm(d_model)
        self.mlp = nn.Sequential(nn.Linear(d_model, 4*d_model), nn.GELU(), nn.Linear(4*d_model, d_model))
    def forward(self,x):
        x = x + self.attn(self.ln1(x)); x = x + self.mlp(self.ln2(x)); return x

class EceGPT(nn.Module):
    def __init__(self, vocab_size, d_model=128, n_layers=3, n_heads=4, max_len=256):
        super().__init__()
        self.tok = nn.Embedding(vocab_size, d_model); self.pos = nn.Embedding(max_len, d_model)
        self.blocks = nn.ModuleList([Block(d_model, n_heads) for _ in range(n_layers)])
        self.lnf = nn.LayerNorm(d_model); self.head = nn.Linear(d_model, vocab_size, bias=False)
        self.max_len = max_len
    def forward(self, idx):
        B,T = idx.size(); pos = torch.arange(T, device=idx.device).unsqueeze(0).expand(B,T)
        x = self.tok(idx) + self.pos(pos)
        for b in self.blocks: x = b(x)
        x = self.lnf(x); return self.head(x)


## 4) Hızlı eğitim

In [ ]:

train_loader = DataLoader(LMDataset(train_texts), batch_size=16, shuffle=True, collate_fn=collate)
val_loader   = DataLoader(LMDataset(val_texts), batch_size=16, shuffle=False, collate_fn=collate)

model = EceGPT(V, d_model=128, n_layers=3, n_heads=4, max_len=256).to(device)
opt = torch.optim.AdamW(model.parameters(), lr=3e-4)

def eval_loss(loader):
    model.eval(); tot=0; n=0
    with torch.no_grad():
        for x,y in loader:
            logits = model(x)
            loss = F.cross_entropy(logits.view(-1, V), y.view(-1), ignore_index=0)
            tot += loss.item(); n += 1
    model.train(); return tot/max(n,1)

for step,(x,y) in enumerate(train_loader,1):
    logits = model(x)
    loss = F.cross_entropy(logits.view(-1, V), y.view(-1), ignore_index=0)
    opt.zero_grad(); loss.backward(); torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0); opt.step()
    if step % 50 == 0: print(f"step {step:03d} loss {loss.item():.3f} val {eval_loss(val_loader):.3f}")
    if step >= 300: break

torch.save(model.state_dict(), "ece_bank_model_qwks.pt")


## 5) Üretim yardımcıları — K, V, S üret

In [ ]:

def generate(model, prompt: str, max_new_tokens=80, top_k=8, min_length=20):
    model.eval()
    ids = torch.tensor(encode(prompt), dtype=torch.long, device=device).unsqueeze(0)
    with torch.no_grad():
        for _ in range(max_new_tokens):
            if ids.size(1) >= model.max_len: ids = ids[:, -model.max_len:]
            logits = model(ids)[:, -1, :]
            vals, idxs = torch.topk(logits, k=min(top_k, logits.size(-1)))
            probs = torch.softmax(vals, dim=-1)
            nxt = idxs.gather(-1, torch.multinomial(probs, 1))
            ids = torch.cat([ids, nxt], dim=1)
            if ids.size(1) >= min_length and itos[int(nxt)] == "<EOS>": break
    return decode(ids[0].tolist())

def extract_between(text, start_tag, end_tag):
    try:
        s = text.split(start_tag, 1)[1]; s = s.split(end_tag, 1)[0]
        return s.strip()
    except Exception:
        return ""

def produce_KVS(Q: str, W: str):
    prompt = f"<EVQ> {Q} </EVQ> <EVW> {W} </EVW> <EVK>"
    gen = generate(model, prompt, max_new_tokens=100, top_k=8, min_length=25)
    K = extract_between(gen, "<EVK>", "</EVK>") or "kredi talebi artarsa"
    V = extract_between(gen, "<EVV>", "</EVV>") or "karlılık azalır"
    S = extract_between(gen, "<EVS>", "</EVS>") or "bankalar temkinli olur"
    return K, V, S, gen

produce_KVS("Sermaye yeterliliği", "kredi portföyü hızla genişlerse")


## 6) Banka state makinesi — model tabanlı K/V/S

In [ ]:

@dataclass
class BankState:
    step: int = 0
    cash_reserve: float = 1000.0
    deposits: float = 10000.0
    loans: float = 8000.0
    npl_ratio: float = 0.03
    capital: float = 1200.0
    liquidity_buffer: float = 1500.0
    net_interest_margin: float = 0.035
    operating_cost: float = 200.0
    rating: float = 0.70
    confidence: float = 0.75
    def snapshot(self) -> Dict:
        return asdict(self)

@dataclass
class EventQWKS:
    Q: str; W: str; K: str; V: str; S: str; gen_text: str

def apply_rules(state: BankState, ev: EventQWKS) -> BankState:
    s = BankState(**state.snapshot())
    w_impact = 1.0; k_impact = 1.0

    if "mevduat çıkışı" in ev.W:
        outflow = 0.03 * s.deposits * w_impact
        s.deposits -= outflow; s.cash_reserve -= 0.5 * outflow; s.liquidity_buffer -= 0.2 * outflow; s.confidence -= 0.02
    if "faizler yükselir" in ev.W:
        s.net_interest_margin += 0.002 * w_impact; s.confidence -= 0.005
    if "faizler düşer" in ev.W:
        s.net_interest_margin -= 0.0015 * w_impact; s.confidence += 0.005; s.loans *= (1 + 0.01 * w_impact)
    if "kredi portföyü hızla genişlerse" in ev.W:
        s.loans *= 1.02; s.npl_ratio += 0.001 * w_impact; s.confidence += 0.003
    if "ekonomik belirsizlik artarsa" in ev.W:
        s.confidence -= 0.02; s.npl_ratio += 0.002 * w_impact

    if "karşılık giderleri yükselirse" in ev.K:
        provision = 0.003 * s.loans * k_impact
        s.capital -= 0.5 * provision; s.cash_reserve -= 0.5 * provision; s.npl_ratio += 0.001
    if "kısa vadeli borçlanma maliyeti artarsa" in ev.K:
        s.net_interest_margin -= 0.002 * k_impact; s.liquidity_buffer -= 0.05 * s.liquidity_buffer
    if "kredi talebi artarsa" in ev.K:
        s.loans *= 1.01; s.confidence += 0.004
    if "vade uyumsuzluğu büyürse" in ev.K:
        s.liquidity_buffer -= 0.1 * s.liquidity_buffer; s.rating -= 0.02
    if "sendikasyon yenilenmezse" in ev.K:
        s.liquidity_buffer -= 0.08 * s.liquidity_buffer; s.confidence -= 0.03

    if "sermaye oranı geriler" in ev.V:
        s.capital *= 0.98; s.rating -= 0.01
    if "likidite tamponu erir" in ev.V:
        s.liquidity_buffer *= 0.95; s.rating -= 0.015
    if "karlılık azalır" in ev.V:
        s.operating_cost *= 1.02; s.net_interest_margin -= 0.001
    if "kredi arzı kısılır" in ev.V:
        s.loans *= 0.995; s.confidence -= 0.01
    if "varlık kalitesi bozulur" in ev.V:
        s.npl_ratio += 0.002; s.rating -= 0.01

    if "sermaye artırımı" in ev.S:
        injection = 200.0; s.capital += injection; s.confidence += 0.03
    if "bilançoda küçülme" in ev.S or "bilançoda küçülmeye gidilir" in ev.S:
        s.loans *= 0.99; s.deposits *= 0.995; s.liquidity_buffer *= 1.02
    if "faiz riski yönetimi sıkılaşır" in ev.S:
        s.net_interest_margin -= 0.0005; s.rating += 0.01
    if "ödeme gücü riske girer" in ev.S:
        s.confidence -= 0.03; s.liquidity_buffer *= 0.98
    if "bankalar temkinli olur" in ev.S:
        s.loans *= 0.998; s.confidence -= 0.005

    interest_income = s.net_interest_margin * s.loans / 4.0
    profit = interest_income - s.operating_cost * 0.25
    s.capital += 0.4 * profit

    s.npl_ratio = max(0.0, min(0.25, s.npl_ratio))
    s.net_interest_margin = max(-0.01, min(0.10, s.net_interest_margin))
    s.confidence = max(0.0, min(1.0, s.confidence))
    s.rating = max(0.0, min(1.0, s.rating))
    return s

def trigger_next_Q(state: BankState, last_event: EventQWKS) -> str:
    if state.liquidity_buffer < 0.9 * 1500 or "likidite" in last_event.V:
        return "Likidite yönetimi"
    if state.npl_ratio > 0.05 or "varlık kalitesi" in last_event.V:
        return "Varlık kalitesi"
    if state.confidence < 0.6:
        return "Mevduat tabanı"
    if state.capital < 1000:
        return "Sermaye yeterliliği"
    return random.choice(["Kredi büyümesi", "Faiz riski", "Piyasa riski"])

def simulate(steps=12):
    history = []; state = BankState(); Q = "Sermaye yeterliliği"
    for t in range(1, steps+1):
        W = "kredi portföyü hızla genişlerse" if t==1 else random.choice([
            "mevduat çıkışı hızlanırsa","faizler yükselirse","faizler düşerse","ekonomik belirsizlik artarsa"
        ])
        K,V,S,gen = produce_KVS(Q,W)
        ev = EventQWKS(Q,W,K,V,S,gen)
        state = apply_rules(state, ev); state.step = t
        history.append({
            "t": t, "Q": Q, "W": W, "K": K, "V": V, "S": S, "gen_text": gen,
            **{k:v for k,v in state.snapshot().items() if k not in ["step"]}
        })
        Q = trigger_next_Q(state, ev)
    return pd.DataFrame(history)

df_hist = simulate(steps=12)
df_hist.head()


## 7) Grafik Rapor ve CSV

In [ ]:

os.makedirs("artifacts", exist_ok=True)
df_hist.to_csv("artifacts/bank_sim_with_model.csv", index=False)

plt.figure(figsize=(8,4)); plt.plot(df_hist["t"], df_hist["capital"]); plt.title("Sermaye (Capital)"); plt.xlabel("Adım"); plt.ylabel("Miktar"); plt.tight_layout(); plt.show()
plt.figure(figsize=(8,4)); plt.plot(df_hist["t"], df_hist["liquidity_buffer"]); plt.title("Likidite Tamponu"); plt.xlabel("Adım"); plt.ylabel("Miktar"); plt.tight_layout(); plt.show()
plt.figure(figsize=(8,4)); plt.plot(df_hist["t"], df_hist["npl_ratio"]); plt.title("NPL Oranı"); plt.xlabel("Adım"); plt.ylabel("Oran"); plt.tight_layout(); plt.show()
plt.figure(figsize=(8,4)); plt.plot(df_hist["t"], df_hist["net_interest_margin"]); plt.title("Net Faiz Marjı"); plt.xlabel("Adım"); plt.ylabel("Oran"); plt.tight_layout(); plt.show()
plt.figure(figsize=(8,4)); plt.plot(df_hist["t"], df_hist["confidence"]); plt.title("Güven (Confidence)"); plt.xlabel("Adım"); plt.ylabel("Skor"); plt.tight_layout(); plt.show()

"artifacts/bank_sim_with_model.csv"
